# CYBER Master

This project is to process the data of recorded voices and baseline measurements for the attachment style interaction with depression and anxiety, in a sense this will work a the ETL or pre-processing of the data.

This notebook has a single purpose expressed in the following steps:
- Gather the answers from the survey DB
- Process the answers by uniques and present a report of the demographics, attachment styles and HADS answers
- From each answer, get the recordings from blob storage.
- For each recording transcribe them and associate them to the answer
- Store the resulting data in the DB for further analysis

In [8]:
# -*- coding: utf-8 -*-

# Import the necessary libraries
import os
import sys
import json
from dotenv import load_dotenv

load_dotenv(find_dotenv())

CONSENT_SURVEY_ID = "8"
STUDY_SURVEY_ID = "7"

## Connect to the DB

In [11]:
for name, value in os.environ.items():
    print(f"{name}={value}")

print(os.environ.get("DB_USER"))

SHELL=/bin/bash
NVM_INC=/home/dvelez/.nvm/versions/node/v20.3.1/include/node
WSL2_GUI_APPS_ENABLED=1
CONDA_EXE=/home/dvelez/miniconda3/bin/conda
_CE_M=
WSL_DISTRO_NAME=Ubuntu-20.04
POETRY_ACTIVE=1
NAME=LAPTOP-TVLSUB7V
PWD=/mnt/d/CYBER/Valencia/dev/attachment-jupyter
LOGNAME=dvelez
MOTD_SHOWN=update-motd
HOME=/home/dvelez
LANG=C.UTF-8
WSL_INTEROP=/run/WSL/634_interop
LS_COLORS=rs=0:di=01;34:ln=01;36:mh=00:pi=40;33:so=01;35:do=01;35:bd=40;33;01:cd=40;33;01:or=40;31;01:mi=00:su=37;41:sg=30;43:ca=30;41:tw=30;42:ow=34;42:st=37;44:ex=01;32:*.tar=01;31:*.tgz=01;31:*.arc=01;31:*.arj=01;31:*.taz=01;31:*.lha=01;31:*.lz4=01;31:*.lzh=01;31:*.lzma=01;31:*.tlz=01;31:*.txz=01;31:*.tzo=01;31:*.t7z=01;31:*.zip=01;31:*.z=01;31:*.dz=01;31:*.gz=01;31:*.lrz=01;31:*.lz=01;31:*.lzo=01;31:*.xz=01;31:*.zst=01;31:*.tzst=01;31:*.bz2=01;31:*.bz=01;31:*.tbz=01;31:*.tbz2=01;31:*.tz=01;31:*.deb=01;31:*.rpm=01;31:*.jar=01;31:*.war=01;31:*.ear=01;31:*.sar=01;31:*.rar=01;31:*.alz=01;31:*.ace=01;31:*.zoo=01;31:*.cpio=01

In [20]:
import mysql.connector

connection = mysql.connector.connect(user=os.getenv("DB_USERNAME"), password=os.getenv("DB_PASSWORD"), host=os.getenv("DB_HOST"), database=os.getenv("DB_NAME"), port=os.getenv("DB_PORT"))
cursor = connection.cursor()

# query the answers table
query = f"SELECT * FROM surveys_answer where surveys_answer.survey_id = {STUDY_SURVEY_ID}"
cursor.execute(query)
answers = cursor.fetchall()

In [27]:
json_answers = []

for answer in answers:
    dict_answer = json.loads(json.loads(answer[1]))
    if dict_answer not in json_answers:
        json_answers.append(json.loads(json.loads(answer[1])))

answers_length = len(json_answers)

In [33]:
full_answers = [answer for answer in json_answers if len(answer.keys()) == 61]
len(full_answers)

8

In [35]:
language_set = set([answer["language"] for answer in full_answers])
print(language_set)

{'es'}
